# LangSmith Demo
    By Aaron Roberts

* LangSmith is a platform for building production-grade LLM applications. URL: https://smith.langchain.com/
* It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework
* It seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.

    ** It is in a private Beta and need to be added to wait list (I got an account almost instantly) **

### Set Environment Variables
All you need to do in order to use LangSmith is to set a number of Environment Variables

In [11]:
import os

#LangSmith API keys:
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY') # API key on LangSmith
os.environ['LANGCHAIN_TRACING_V2'] = 'true' # Enable Tracing
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com" # Cloud end point
os.environ['LANGCHAIN_PROJECT'] = "Agent_Test" # Project name

# Other API KEYS:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['SERPAPI_API_KEY'] = os.getenv('SERPAPI_API_KEY') # For internet searches

### Setup the LangSmith API

In [12]:
from langsmith import Client

client = Client() #generate a client object so we can upload datasets and run tests

### Build a Basic Agent

In [13]:
from operator import itemgetter 

from langchain import hub #This is a prompt repository
from langchain.agents import AgentExecutor, load_tools #Loads an Agent with tool capabilites, a way to load in tools by name
from langchain.agents.format_scratchpad import format_to_openai_functions # Convert Agent Actions and Tool outputs into function messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser # Parses a message into an Agent Action or the final output
from langchain.chat_models import ChatOpenAI #This is the basic chat API for OpenAI
from langchain.tools.render import format_tool_to_openai_function #Converts tool to match the openAI function

llm = ChatOpenAI(temperature=0) # Creates a llm connection 3.5 turbo by default
tools = load_tools(["serpapi", "llm-math"], llm=llm) # Load in the tools
llm_with_tools = llm.bind(functions = [format_tool_to_openai_function(t) for t in tools]) # Adding the tools as OpenAI functions

prompt = hub.pull("wfh/agent-lcel-prompt") # Pulls a prompt from langhub that works well with agents with tools


#Builds out the agent using LCEL (langChain expression language)
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_functions(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

# Build the Agent RunTime
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    handle_parsing_errors=True,
    return_intermediate_steps=True
)

### Create Some Inputs for the Agent

In [14]:
inputs = [
    "What is the number of cats in the US multiplied by the number of cats in China",
    "How much is all of the cats in the world multiplied by the number of the number of legs that a cat has?",
    "How many cats are there in the US divided by the number of US citizens?",
    "How much larger is an average cat to an average dog?",
    "Are there more dogs or cats in the US?",
    "What is the size ratio from the largest cat to the smallest cat?",
    "How many cats does it take to fill a 1500 square foot house?",
    "How many more hours per day do cats sleep than the average human?",
    "How many cat themed Gods are there compared to dog themed Gods?",
    "Is the largest rat bigger than the smallest cat?"
]

### Batch the inputs and send them langsmith.

In [15]:
results = agent_executor.batch([{"input": x} for x in inputs], return_exceptions=True)

In [7]:
agent_executor.invoke({"input":"s the largest rat bigger than the smallest cat?"})

{'input': 's the largest rat bigger than the smallest cat?',
 'output': 'The largest rat, known as the Gambian pouched rat, can grow up to 3 feet long including the tail. On the other hand, the smallest cat breed, the Singapura, typically weighs around 4-8 pounds and is about 8-10 inches tall. So, the largest rat is generally bigger than the smallest cat.',
 'intermediate_steps': []}